# PREPARE THE SYSTEM

In [3]:
import simtk.openmm as mm       # Main OpenMM functionality
import simtk.openmm.app as app  # Application layer (handy interface)
import simtk.unit as unit       # Unit/quantity handling
import os

### (a) Saving the first structure to a new .pdb file

In [4]:
cwd = os.getcwd()                                         # Current working directory path
pdb_file = os.path.join(cwd, 'files/6a5j_protein/6a5j.pdb')        # pdb file path
new_pdb_file = os.path.join(cwd, 'files/6a5j_protein/conf.pdb')    # new pdb file path

with open(pdb_file) as pdbfile:
    with open(new_pdb_file, 'w') as newfile:             
        model_count = 0
        for line in pdbfile:
            if line.startswith("MODEL"):
                model_count += 1
                if model_count > 1:
                    break
                continue
            if not line.startswith("ATOM"):
                continue
            newfile.write(line)
            
# This frame will be used as a starting structure to the simulation

### (b) Topology of the conf.pdb

In [5]:
molecule = app.PDBFile(new_pdb_file)
print(molecule.topology)

<Topology; 1 chains, 13 residues, 260 atoms, 259 bonds>


### (c) Molecule simulation using the AMBER 14SB force field in TIP3P water

In [6]:
forcefield = app.ForceField(
    "amber14/protein.ff14SB.xml",          # Solute
    "amber14/tip3p.xml"                    # Solvent
)

In [7]:
# The molecule is placed in a cubic water box with a minimum distance of 1 nm to the box borders

model = app.Modeller(molecule.topology, molecule.positions)
model.addSolvent(
    forcefield,
    padding=1 * unit.nanometer,        
    neutralize=True                  # Neutralize the system
    )

In [9]:
molecule.positions = model.getPositions()     
molecule.topology = model.getTopology()

solvent_solute_system = os.path.join(cwd, 'files/6a5j_protein/solv.pdb')

with open(solvent_solute_system, "w") as file_:
    molecule.writeFile(
        molecule.topology, molecule.positions,
        file=file_
    )
    
# One can visulaize this system in VMD